# DenseNet : Optimization

In [29]:
import math
import numpy as np
import tensorflow as tf
import keras
from keras import applications, Model
from keras.models import Sequential
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Dropout, Flatten, Dense, BatchNormalization
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import RandomizedSearchCV, KFold
from PIL import Image
import os
import matplotlib.pyplot as plt
from keras.optimizers import Adam
from keras.preprocessing.image import img_to_array
from sklearn.metrics import make_scorer
from keras.utils.np_utils import to_categorical
import random
import sklearn
import pandas as pd
import warnings

warnings.simplefilter('ignore')

In [30]:
train_data_dir = 'Data/Training_Data'
validation_data_dir = 'Data/Validation_Data'
test_data_dir= 'Data/Test_Data'

training_features_file = 'Features/training_features_Densenet.npy'
validation_features_file = 'Features/validation_features_Densenet.npy'
top_weights_file = 'Weights/weights_Densenet.h5'
model_file = 'Models/model_Densenet.h5'

train_labels_file = 'Labels/training_labels.npy'
validation_labels_file = 'Labels/validation_labels.npy'
test_labels_file = 'Labels/test_labels.npy'

img_width, img_height = 224, 224
NB_CLASSES = 11

In [31]:
train_labels = np.load(open(train_labels_file, 'rb'))
validation_labels = np.load(open(validation_labels_file, 'rb'))
test_labels = np.load(open(test_labels_file, 'rb'))

In [32]:
print('Training Data : ' + str(len(train_labels)) + ' Images')
print('Validation Data : ' + str(len(validation_labels)) + ' Images')
print('Test Data : ' + str(len(test_labels)) + ' Images')

Training Data : 307 Images
Validation Data : 74 Images
Test Data : 18 Images


## Converting images to feature vectors using weights from ImageNet

In [33]:
def images_to_feature_vectors(model, directory, batch_size, steps):
    
    datagen = ImageDataGenerator()
    
    generator = datagen.flow_from_directory(
        directory,
        target_size=(img_width, img_height),
        batch_size=batch_size,
        class_mode=None,
        shuffle=False) # Keep the data in the same order
    
    features = model.predict_generator(generator, steps, verbose=1) 
    
    return features

In [34]:
# Batch size has to be a multiple of the number of images  to keep our vectors consistents
training_batch_size = 1 # batch size for feature pre-training
validation_batch_size = 1 # batch size for feature pre-training

model = applications.DenseNet121(include_top=False, weights='imagenet', input_shape=(img_width,img_height,3))
training_features = images_to_feature_vectors(model, train_data_dir, training_batch_size, len(train_labels) // training_batch_size)
validation_features = images_to_feature_vectors(model, validation_data_dir, validation_batch_size, len(validation_labels) // validation_batch_size)

Found 307 images belonging to 11 classes.
307/307 [==============================] - 46s 149ms/step
Found 74 images belonging to 11 classes.
74/74 [==============================] - 10s 132ms/step


In [35]:
with open(training_features_file, 'wb') as file:
        np.save(file, training_features, allow_pickle = False)
with open(validation_features_file, 'wb') as file:
        np.save(file, validation_features, allow_pickle = False)

## Randomized Search CV training

In [36]:
def create_model(lr, decay, nn1, nn2, nn3, input_shape, output_shape):
    '''This is a model generating function so that we can search over neural net 
    parameters and architecture'''
    
    opt = keras.optimizers.Adam(lr=lr, decay=decay)
                                                     
    model = Sequential()
    
    model.add(Flatten())
    model.add(Dense(nn1, input_dim = input_shape, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dense(nn2, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dense(nn3, activation='relu'))
    model.add(BatchNormalization())
            
    model.add(Dense(output_shape, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'],)
    return model

In [37]:
model = KerasClassifier(build_fn=create_model, epochs=16, verbose=1)

In [38]:
# Learning rate values
lr=[1e-2, 1e-3, 1e-4]
decay=[1e-6,1e-9,0]

# Number of neurons per layer
nn1=[4096,2048,1024]
nn2=[2048,1024,512]
nn3=[1000,500,200]

batch_size=[2048,1024,512]

In [39]:
train_data = np.load(open(training_features_file, 'rb'))
#train_data = train_data.reshape(train_data.shape[0],-1)
validation_data = np.load(open(validation_features_file, 'rb'))
#validation_data = validation_data.reshape(validation_data.shape[0],-1)
    
train_labels_onehot = to_categorical(train_labels,NB_CLASSES)            #One Hot Encoder
validation_labels_onehot = to_categorical(validation_labels,NB_CLASSES)  #One Hot Encoder

In [40]:
# dictionary summary
param_grid = dict(
                    lr=lr, decay=decay, nn1=nn1, nn2=nn2, nn3=nn3,
                    batch_size=batch_size,
                    input_shape=train_data.shape[1:], output_shape = (NB_CLASSES,)
                 )


grid = RandomizedSearchCV(estimator=model, cv=KFold(3), param_distributions=param_grid, 
                          verbose=20,  n_iter=10, n_jobs=1)

In [13]:
grid_result = grid.fit(train_data, train_labels_onehot)

Fitting 3 folds for each of 10 candidates, totalling 30 fits
[CV] output_shape=101, nn3=500, nn2=512, nn1=4096, lr=0.001, input_shape=7, decay=1e-06, batch_size=2048 


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


Instructions for updating:
Use tf.cast instead.
Epoch 1/16
2114/2114 [==============================] - 20s 10ms/step - loss: 5.4102 - acc: 0.0180
Epoch 2/16
2114/2114 [==============================] - 9s 4ms/step - loss: 2.9312 - acc: 0.3444
Epoch 3/16
2114/2114 [==============================] - 9s 4ms/step - loss: 2.3313 - acc: 0.4551
Epoch 4/16
2114/2114 [==============================] - 9s 4ms/step - loss: 2.0470 - acc: 0.5303
Epoch 5/16
2114/2114 [==============================] - 9s 4ms/step - loss: 1.8616 - acc: 0.5667
Epoch 6/16
2114/2114 [==============================] - 9s 4ms/step - loss: 1.6602 - acc: 0.6244
Epoch 7/16
2114/2114 [==============================] - 9s 4ms/step - loss: 1.4853 - acc: 0.6679
Epoch 8/16
2114/2114 [==============================] - 9s 4ms/step - loss: 1.3640 - acc: 0.6958
Epoch 9/16
2114/2114 [==============================] - 9s 4ms/step - loss: 1.2373 - acc: 0.7190
Epoch 10/16
2114/2114 [==============================] - 9s 4ms/step - loss: 

[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:  2.7min remaining:    0.0s


Epoch 1/16
2114/2114 [==============================] - 20s 9ms/step - loss: 5.3678 - acc: 0.0189
Epoch 2/16
2114/2114 [==============================] - 9s 4ms/step - loss: 2.5143 - acc: 0.4513
Epoch 3/16
2114/2114 [==============================] - 9s 4ms/step - loss: 1.9695 - acc: 0.5643
Epoch 4/16
2114/2114 [==============================] - 9s 4ms/step - loss: 1.7356 - acc: 0.6154
Epoch 5/16
2114/2114 [==============================] - 9s 4ms/step - loss: 1.5869 - acc: 0.6481
Epoch 6/16
2114/2114 [==============================] - 9s 4ms/step - loss: 1.4457 - acc: 0.6798
Epoch 7/16
2114/2114 [==============================] - 9s 4ms/step - loss: 1.3186 - acc: 0.7039
Epoch 8/16
2114/2114 [==============================] - 9s 4ms/step - loss: 1.1868 - acc: 0.7266
Epoch 9/16
2114/2114 [==============================] - 9s 4ms/step - loss: 1.0620 - acc: 0.7621
Epoch 10/16
2114/2114 [==============================] - 9s 4ms/step - loss: 0.9516 - acc: 0.7867
Epoch 11/16
2114/2114 [=====

...

[Parallel(n_jobs=1)]: Done  19 out of  19 | elapsed: 46.8min remaining:    0.0s


Epoch 1/16
2114/2114 [==============================] - 27s 13ms/step - loss: 4.3527 - acc: 0.1599
Epoch 2/16
2114/2114 [==============================] - 12s 6ms/step - loss: 2.0333 - acc: 0.5307
Epoch 3/16
2114/2114 [==============================] - 12s 6ms/step - loss: 1.4647 - acc: 0.6547
Epoch 4/16
2114/2114 [==============================] - 12s 6ms/step - loss: 1.1457 - acc: 0.7271
Epoch 5/16
2114/2114 [==============================] - 12s 6ms/step - loss: 0.9471 - acc: 0.7777
Epoch 6/16
2114/2114 [==============================] - 12s 6ms/step - loss: 0.7353 - acc: 0.8278
Epoch 7/16
2114/2114 [==============================] - 12s 6ms/step - loss: 0.5695 - acc: 0.8756
Epoch 8/16
2114/2114 [==============================] - 12s 5ms/step - loss: 0.4728 - acc: 0.9016
Epoch 9/16
2114/2114 [==============================] - 12s 6ms/step - loss: 0.3925 - acc: 0.9229
Epoch 10/16
2114/2114 [==============================] - 12s 6ms/step - loss: 0.3161 - acc: 0.9376
Epoch 11/16
2114/2

1057/1057 [==============================] - 5s 4ms/step
[CV]  output_shape=101, nn3=500, nn2=1024, nn1=2048, lr=0.001, input_shape=7, decay=1e-06, batch_size=1024, score=0.000, total= 2.1min
[CV] output_shape=101, nn3=500, nn2=1024, nn1=2048, lr=0.001, input_shape=7, decay=1e-06, batch_size=1024 
Epoch 1/16
2114/2114 [==============================] - 21s 10ms/step - loss: 4.5174 - acc: 0.1310
Epoch 2/16
2114/2114 [==============================] - 7s 3ms/step - loss: 2.1739 - acc: 0.5095
Epoch 3/16
2114/2114 [==============================] - 7s 3ms/step - loss: 1.5155 - acc: 0.6429
Epoch 4/16
2114/2114 [==============================] - 7s 3ms/step - loss: 1.1743 - acc: 0.7375
Epoch 5/16
2114/2114 [==============================] - 7s 3ms/step - loss: 0.9387 - acc: 0.7886
Epoch 6/16
2114/2114 [==============================] - 7s 3ms/step - loss: 0.7247 - acc: 0.8510
Epoch 7/16
2114/2114 [==============================] - 7s 3ms/step - loss: 0.5808 - acc: 0.8746
Epoch 8/16
2114/2114

[Parallel(n_jobs=1)]: Done  30 out of  30 | elapsed: 67.7min finished


Epoch 1/16
3171/3171 [==============================] - 24s 7ms/step - loss: 4.7408 - acc: 0.0703
Epoch 2/16
3171/3171 [==============================] - 9s 3ms/step - loss: 2.0763 - acc: 0.5762
Epoch 3/16
3171/3171 [==============================] - 9s 3ms/step - loss: 1.3086 - acc: 0.7871
Epoch 4/16
3171/3171 [==============================] - 9s 3ms/step - loss: 0.8422 - acc: 0.8953
Epoch 5/16
3171/3171 [==============================] - 9s 3ms/step - loss: 0.5437 - acc: 0.9533
Epoch 6/16
3171/3171 [==============================] - 9s 3ms/step - loss: 0.3475 - acc: 0.9814
Epoch 7/16
3171/3171 [==============================] - 9s 3ms/step - loss: 0.2327 - acc: 0.9940
Epoch 8/16
3171/3171 [==============================] - 9s 3ms/step - loss: 0.1521 - acc: 0.9981
Epoch 9/16
3171/3171 [==============================] - 9s 3ms/step - loss: 0.1036 - acc: 0.9994
Epoch 10/16
3171/3171 [==============================] - 9s 3ms/step - loss: 0.0740 - acc: 1.0000
Epoch 11/16
3171/3171 [=====

In [14]:
cv_results_df = pd.DataFrame(grid_result.cv_results_)
cv_results_df.to_csv('gridsearch_Inception.csv')

In [15]:
cv_results_df

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_output_shape,param_nn3,param_nn2,param_nn1,param_lr,param_input_shape,param_decay,param_batch_size,params,split0_test_score,split1_test_score,split2_test_score,mean_test_score,std_test_score,rank_test_score
0,156.063100,0.842942,2.285238,0.037467,101,500,512,4096,0.001,7,1e-06,2048,"{'output_shape': 101, 'nn3': 500, 'nn2': 512, ...",0.000946,0.002838,0.002838,0.002208,0.000892,2
1,89.610148,1.547271,2.102973,0.107218,101,200,512,1024,0.001,7,1e-09,512,"{'output_shape': 101, 'nn3': 200, 'nn2': 512, ...",0.000000,0.000946,0.001892,0.000946,0.000772,6
2,298.679060,2.168388,3.665428,0.196830,101,500,1024,4096,0.001,1024,1e-09,512,"{'output_shape': 101, 'nn3': 500, 'nn2': 1024,...",0.000000,0.000946,0.000946,0.000631,0.000446,10
3,119.893126,0.285009,2.857568,0.124893,101,500,2048,2048,0.001,1024,1e-06,1024,"{'output_shape': 101, 'nn3': 500, 'nn2': 2048,...",0.000000,0.000946,0.001892,0.000946,0.000772,6
4,67.929503,0.982511,3.050566,0.333164,101,200,512,1024,0.0001,7,1e-09,1024,"{'output_shape': 101, 'nn3': 200, 'nn2': 512, ...",0.002838,0.000000,0.001892,0.001577,0.001180,4
5,115.734195,0.430777,3.421132,0.131229,101,500,1024,2048,0.0001,1024,1e-09,1024,"{'output_shape': 101, 'nn3': 500, 'nn2': 1024,...",0.000946,0.003784,0.001892,0.002208,0.001180,1
6,205.122791,1.588669,4.554182,0.141909,101,1000,512,4096,0.001,1024,0,1024,"{'output_shape': 101, 'nn3': 1000, 'nn2': 512,...",0.001892,0.000000,0.002838,0.001577,0.001180,3
7,72.048003,0.677030,4.078477,0.144334,101,200,1024,1024,0.001,1024,1e-06,1024,"{'output_shape': 101, 'nn3': 200, 'nn2': 1024,...",0.000000,0.000000,0.002838,0.000946,0.001338,8
8,73.390035,1.318049,4.293865,0.150274,101,500,1024,1024,0.001,7,0,1024,"{'output_shape': 101, 'nn3': 500, 'nn2': 1024,...",0.000000,0.000000,0.002838,0.000946,0.001338,8
9,120.233964,1.742860,4.846889,0.212204,101,500,1024,2048,0.001,7,1e-06,1024,"{'output_shape': 101, 'nn3': 500, 'nn2': 1024,...",0.000000,0.001892,0.001892,0.001261,0.000892,5


In [16]:
grid_result.best_params_

{'output_shape': 101,
 'nn3': 500,
 'nn2': 1024,
 'nn1': 2048,
 'lr': 0.0001,
 'input_shape': 1024,
 'decay': 1e-09,
 'batch_size': 1024}

In [41]:
# Parameters after RandomizedSearchCV

nn1 = 2048; nn2 = 1024; nn3 = 500
lr = 0.0001; decay=1e-09
batch_size = 1024

## Regularization

In [42]:
# Regularzation Parameters

dropout = 0.5
l1 = 0.0001
l2 = 0.0001

In [43]:
def model():
    opt = keras.optimizers.Adam(lr=lr)
    reg = keras.regularizers.l1_l2(l1=l1, l2=l2)
                                                     
    model = Sequential()
    
    model.add(Flatten())
    model.add(Dense(nn1, activation='relu', kernel_regularizer=reg))
    model.add(Dropout(dropout))
    model.add(BatchNormalization())
    model.add(Dense(nn2, activation='relu', kernel_regularizer=reg))
    model.add(Dropout(dropout))
    model.add(BatchNormalization())
    model.add(Dense(nn3, activation='relu', kernel_regularizer=reg))
    model.add(Dropout(dropout))
    model.add(BatchNormalization())
            
    model.add(Dense(NB_CLASSES, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'],)
    return model

In [44]:
train_labels_onehot = to_categorical(train_labels, NB_CLASSES)            #One Hot Encoder
validation_labels_onehot = to_categorical(validation_labels, NB_CLASSES)  #One Hot Encoder

In [45]:
top = model()

In [46]:
top.fit(train_data, train_labels_onehot,
              epochs=64,
              batch_size=batch_size,
              validation_data=(validation_data, validation_labels_onehot))

Train on 307 samples, validate on 74 samples
Epoch 1/64
307/307 [==============================] - 32s 103ms/step - loss: 65.1928 - acc: 0.0912 - val_loss: 63.6078 - val_acc: 0.2297
Epoch 2/64
307/307 [==============================] - 3s 11ms/step - loss: 64.1822 - acc: 0.1368 - val_loss: 62.6320 - val_acc: 0.4189
Epoch 3/64
307/307 [==============================] - 3s 10ms/step - loss: 63.3278 - acc: 0.2117 - val_loss: 61.9100 - val_acc: 0.4730
Epoch 4/64
307/307 [==============================] - 3s 11ms/step - loss: 62.4293 - acc: 0.2443 - val_loss: 61.2190 - val_acc: 0.5405
Epoch 5/64
307/307 [==============================] - 3s 11ms/step - loss: 61.6148 - acc: 0.3681 - val_loss: 60.5512 - val_acc: 0.6486
Epoch 6/64
307/307 [==============================] - 3s 11ms/step - loss: 60.8076 - acc: 0.4365 - val_loss: 59.8947 - val_acc: 0.6892
Epoch 7/64
307/307 [==============================] - 3s 10ms/step - loss: 60.3527 - acc: 0.4593 - val_loss: 59.2835 - val_acc: 0.7027
Epoch 8/

Epoch 61/64
307/307 [==============================] - 3s 10ms/step - loss: 33.7005 - acc: 0.9935 - val_loss: 33.7556 - val_acc: 0.8378
Epoch 62/64
307/307 [==============================] - 3s 10ms/step - loss: 33.3478 - acc: 0.9902 - val_loss: 33.4040 - val_acc: 0.8378
Epoch 63/64
307/307 [==============================] - 3s 10ms/step - loss: 32.9685 - acc: 1.0000 - val_loss: 33.0573 - val_acc: 0.8378
Epoch 64/64
307/307 [==============================] - 3s 11ms/step - loss: 32.6412 - acc: 0.9935 - val_loss: 32.7150 - val_acc: 0.8378


In [47]:
top.save_weights(top_weights_file)

## Complete Model

In [48]:
def complete_model():
    opt = keras.optimizers.Adam(lr=lr)
    reg = keras.regularizers.l1_l2(l1=l1, l2=l2)
                                                 
    base = applications.DenseNet121(include_top=False, weights='imagenet', input_shape=(img_width,img_height,3))
    
    top = Sequential()

    top.add(Flatten(input_shape=train_data.shape[1:]))
    top.add(Dense(nn1, activation='relu', kernel_regularizer=reg))
    top.add(BatchNormalization())
    top.add(Dense(nn2, activation='relu', kernel_regularizer=reg))
    top.add(BatchNormalization())
    top.add(Dense(nn3, activation='relu', kernel_regularizer=reg))
    top.add(BatchNormalization())            
    top.add(Dense(NB_CLASSES, activation='softmax'))
    top.load_weights(top_weights_file)
    top.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'],)
    
    
    model = Model(input= base.input, output= top(base.output))
    for layer in model.layers:
        layer.trainable = False
    model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'],)
    return model

In [49]:
model = complete_model()

datagen = ImageDataGenerator()

print("Model Summary")
print(model.summary())

Model Summary
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_4 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
zero_padding2d_7 (ZeroPadding2D (None, 230, 230, 3)  0           input_4[0][0]                    
__________________________________________________________________________________________________
conv1/conv (Conv2D)             (None, 112, 112, 64) 9408        zero_padding2d_7[0][0]           
__________________________________________________________________________________________________
conv1/bn (BatchNormalization)   (None, 112, 112, 64) 256         conv1/conv[0][0]                 
_______________________________________________________________________________________________

In [50]:
generator = datagen.flow_from_directory(
            test_data_dir,
            target_size=(img_width, img_height),
            batch_size=len(test_labels),
            class_mode=None,
            shuffle=False)
test_predictions = model.predict_generator(generator, 1, verbose=1) 

Found 18 images belonging to 11 classes.
1/1 [==============================] - 16s 16s/step


In [51]:
test_predictions = np.asarray(list(map(str,np.argmax(test_predictions,axis=1)))).reshape(-1,1)
test_labels = np.asarray(test_labels).reshape(-1,1)
test_acc, test_acc_op = tf.metrics.accuracy(test_labels, test_predictions)

In [52]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())
sess.run(tf.local_variables_initializer())

In [53]:
test_accuracy = sess.run(test_acc_op)

In [54]:
print('\nTest accuracy : ' + str(test_accuracy))


Test accuracy : 0.8888889


In [55]:
model.save(model_file)